In [2]:
from rich.console import Console
from rich.panel import Panel
from rich.prompt import Prompt
from rich.markdown import Markdown
import openai
import os

console = Console()
messages = [{"role": "system", "content": "Ты полезный ассистент."}]

# Проверяем наличие переменной окружения
api_key = os.getenv("OPENROUTER_API_KEY")

if not api_key:
    # Если переменная не установлена, запрашиваем у пользователя
    api_key = console.input("[bold yellow]Введите API-ключ:[/bold yellow] ").strip()
    
    # Проверяем, что ключ не пустой
    if not api_key:
        console.print(Panel("API-ключ не может быть пустым", style="red"))
        exit(1)

client = openai.OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key
)

def format_message(role, content):
    if role == "user":
        panel = Panel(f"[cyan]{content}[/cyan]", title="Пользователь", border_style="cyan")
    else:
        markdown_content = Markdown(content)
        panel = Panel(markdown_content, title="Ответ", border_style="magenta")
    console.print(panel)

console.print(Panel.fit("Начало сессии. Команды: /exit, /clear, /system", title="Чат", border_style="yellow"))

while True:
    try:
        user_input = Prompt.ask("Ввод").strip()

        if not user_input:
            continue

        if user_input == "/exit":
            console.print(Panel("Завершение работы", title="Статус", expand=False))
            break

        elif user_input == "/clear":
            system_msg = messages[0]
            messages = [system_msg]
            console.print(Panel("История очищена", expand=False))
            continue

        elif user_input.startswith("/system "):
            new_prompt = user_input[8:]
            messages[0] = {"role": "system", "content": new_prompt}
            console.print(Panel(f"Обновлен системный промпт:\n{new_prompt}", expand=False))
            continue

        messages.append({"role": "user", "content": user_input})
        format_message("user", user_input)

        try:
            response = client.chat.completions.create(
                model="qwen/qwen3-coder:free",
                messages=messages,
                temperature=0.7
            )
            bot_response = response.choices[0].message.content.strip()
        except openai.AuthenticationError:
            bot_response = "Ошибка авторизации. Проверьте API-ключ."
        except openai.RateLimitError:
            bot_response = "Превышено количество запросов. Попробуйте позже."
        except Exception as e:
            bot_response = f"Произошла ошибка: {str(e)}"

        messages.append({"role": "assistant", "content": bot_response})
        format_message("assistant", bot_response)

    except KeyboardInterrupt:
        console.print(Panel("Прерывание программы", title="Статус", expand=False))
        break

Введите API-ключ:

╭───────────────────── Чат ──────────────────────╮
│ Начало сессии. Команды: /exit, /clear, /system │
╰────────────────────────────────────────────────╯

Ввод:

╭───────────────────────────────────────────────── Пользователь ──────────────────────────────────────────────────╮
│ Привет                                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── Ответ ─────────────────────────────────────────────────────╮
│ Привет! Как я могу тебе помочь?                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Ввод:

╭───────────────────────────────────────────────── Пользователь ──────────────────────────────────────────────────╮
│ Вау, круто!                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── Ответ ─────────────────────────────────────────────────────╮
│ Спасибо! Рад, что понравилось. Если ещё что-то нужно — будь то рифмы, тексты, идеи или просто поболтать —       │
│ обращайся. Тут всегда можно найти тему для разговора. Как настроение, кстати?                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Ввод:

╭───── Статус ──────╮
│ Завершение работы │
╰───────────────────╯